# Sentiment Analysis with Deep Learning using BERT

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)


## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# connect Google Colab with Google Drive to read file(s) from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
directory_path = '/content/drive/My Drive/CIT/SEMESTER3/I.INTEL/SUB2_Coursera_BERT_SentimentAnalysis'
file_name = 'smileannotationsfinal.csv'

In [4]:
# loading the data set
df = pd.read_csv(os.path.join(directory_path, file_name), names=['id','text','category'])
df.set_index('id', inplace=True)

In [5]:
# having a quick look for the dataset with first 5 rows
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
# printing out one sample text
df.text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [7]:
# examining the counts for each of the values in category
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [8]:
# removing observations having more than 2 feelings, 
# i.e. observations having character "|" in the category
df = df[~df.category.str.contains('\|')]

In [9]:
# removing observations having no feeling, i.e. 'nocode'
df = df[df.category != 'nocode']

In [10]:
# examining again the counts for each of the values in category
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [11]:
# building a dictionary containing keys as feelings in category and
# values being 0,1,2,... accordingly

# firstly creating a list containing all of the keys/values
possible_labels = df.category.unique()
possible_labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [12]:
# then creating a dictionary
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [13]:
# now adding another column of the label that has just been created to the dataframe
df['label'] = df.category.replace(label_dict)
df.head()


,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


## Task 3: Training/Validation Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_val, y_train, y_val = train_test_split(
    # index value - the unique identification for each sample
    df.index.values,

    # label here for randomly splitting the labels
    df.label.values,

    # choosing 15% of the test size
    test_size = 0.15,

    # choosing random state
    random_state=17,

    # because there are more than 2 labels and there is a hugh discrepancy
    # between the number of each of the labels
    # therefore, choosing stratifying random sampling
    # for the splitting training/test sets
    stratify=df.label.values
)

In [16]:
# number of samples
df.shape[0]

1481

In [17]:
# creating a new column 'data_type'
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [18]:
# now setting values 'train' and 'val' for the column 'data_type
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [19]:
# now seeing the results of splitting by the groupby function
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [20]:
# number of samples of training data
len(df[df['data_type'] == 'train'])

1258

In [21]:
# number of samples of test data
len(df[df['data_type'] == 'val'])

223

## Task 4: Loading Tokenizer and Encoding our Data

In [22]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 18.3MB/s 
     |████████████████████████████████| 1.1MB 31.6MB/s 
     |████████████████████████████████| 890kB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2aed5f4deacbd151892dc0337214139ad06be5e8198f5cbc68b335672c8f1209
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [23]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset # this is for setting data usable for pytorch tensor later

In [24]:
tokenizer = BertTokenizer.from_pretrained(
    # choosing uncased version of BERT
    # all of the characters being lower case
    'bert-base-uncased', 

    # thus, converting everything to lower case
    do_lower_case=True)

In [25]:
# FOR TRAINING DATA
encoded_data_train = tokenizer.batch_encode_plus(

    # the actual sentences from the training data as the input
    df[df.data_type == 'train'].text.values,
    
    # for BERT to know where the sentence ends and new one begins 
    add_special_tokens=True,
    
    # because you use the fixed input, we need everything having the same dimensionality
    # in this case, we choose max length 256 in the following argument
    # therefore, when putting attention mask for different sentences having different lengths 
    # the lengths are now having the same leghth will produce 1 or 0 saying where the actual words are
    # 1 will be where the words are, 0 will be not the words 
    return_attention_mask=True,
    
    # pad all sentences to a certain max length
    # then we have to set the following argument max_length a certain value
    pad_to_max_length=True,
    
    # setting the max length of 256 because the tweet is unlikely to be longer than 256 characters
    max_length=256,
    
    # returning pytorch tensors as the outcome
    return_tensors='pt' 
    )

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [26]:
# encoded_data_train behaves like a dict although the printing result is something else
type(encoded_data_train)

transformers.tokenization_utils_base.BatchEncoding

In [27]:
# we can retrieve the keys of encoded_data_train
encoded_data_train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [28]:
# examining the input_ids of encoded_data_train
print(encoded_data_train['input_ids'].shape)
encoded_data_train['input_ids']

torch.Size([1258, 256])


tensor([[  101, 16092,  3897,  ...,     0,     0,     0],
        [  101,  1030, 27034,  ...,     0,     0,     0],
        [  101,  1030, 10682,  ...,     0,     0,     0],
        ...,
        [  101, 11047,  1030,  ...,     0,     0,     0],
        [  101,  1030,  3680,  ...,     0,     0,     0],
        [  101,  1030,  2120,  ...,     0,     0,     0]])

In [29]:
# examining the token_type_ids of encoded_data_train
print(encoded_data_train['token_type_ids'].shape)
encoded_data_train['token_type_ids']

torch.Size([1258, 256])


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [30]:
# examining the attention_mask of encoded_data_train
print(encoded_data_train['attention_mask'].shape)
encoded_data_train['attention_mask']

torch.Size([1258, 256])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [31]:
# input_ids_train -> each word as a number
input_ids_train = encoded_data_train['input_ids']

# the position 1 & 0 where there are words and where there are not
attention_masks_train = encoded_data_train['attention_mask']

# this is just derived from the training dataframe
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

In [32]:
print(input_ids_train.shape)
input_ids_train[0]

torch.Size([1258, 256])


tensor([  101, 16092,  3897,  2007, 10098, 18982,  1001,  2293, 10105,  2015,
         1006,  2013,  1030,  2329,  7606, 14820,  8299,  1024,  1013,  1013,
         1056,  1012,  2522,  1013,  1053,  2549,  2595, 26760,  2140,  2692,
         2229,  2226,  1007,  8299,  1024,  1013,  1013,  1056,  1012,  2522,
         1013,  1044,  2692,  6777, 19279,  2497, 13088,  4160,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [33]:
print(attention_masks_train.shape)
attention_masks_train[0]

torch.Size([1258, 256])


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
print(labels_train.shape)
labels_train

torch.Size([1258])


tensor([0, 0, 0,  ..., 0, 0, 1])

In [35]:
# FOR VALIDATION DATA
encoded_data_val = tokenizer.batch_encode_plus(

    # the actual sentences from the training data as the input
    df[df.data_type == 'val'].text.values,
    
    # for BERT to know where the sentence ends and new one begins 
    add_special_tokens=True,
    
    # because you use the fixed input, we need everything having the same dimensionality
    # in this case, we choose max length 256 in the following argument
    # therefore, when putting attention mask for different sentences having different lengths 
    # the lengths are now having the same leghth will produce 1 or 0 saying where the actual words are
    # 1 will be where the words are, 0 will be not the words 
    return_attention_mask=True,
    
    # pad all sentences to a certain max length
    # then we have to set the following argument max_length a certain value
    pad_to_max_length=True,
    
    # setting the max length of 256 because the tweet is unlikely to be longer than 256 characters
    max_length=256,
    
    # returning pytorch tensors as the outcome
    return_tensors='pt' 
    )

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [36]:
# input_ids_val -> each word as a number
input_ids_val = encoded_data_val['input_ids']

# the position 1 & 0 where there are words and where there are not
attention_masks_val = encoded_data_val['attention_mask']

# this is just derived from the training dataframe
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [37]:
# NOW CREATING TWO DATASETS IN TENSOR FORMS
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            labels_val)

In [38]:
# getting lengths for those two datasets
print(len(dataset_train))
print(len(dataset_val))

1258
223


In [39]:
# getting the type of the datasets of training and validation
print(type(dataset_train))
print(type(dataset_val))

<class 'torch.utils.data.dataset.TensorDataset'>
<class 'torch.utils.data.dataset.TensorDataset'>


In [40]:
# elements in the training or validation dataset has tupple type
print(type(dataset_train[0]))
print(type(dataset_val[0]))

<class 'tuple'>
<class 'tuple'>


In [41]:
# examining number of components in one element
# they include input_ids_train, attention_mask_train & labels_train
len(dataset_train[0])

3

In [42]:
# one element in the dataset_train has 3 components including 
# input_ids, attention_mask & label in that order 
dataset_train[0]

(tensor([  101, 16092,  3897,  2007, 10098, 18982,  1001,  2293, 10105,  2015,
          1006,  2013,  1030,  2329,  7606, 14820,  8299,  1024,  1013,  1013,
          1056,  1012,  2522,  1013,  1053,  2549,  2595, 26760,  2140,  2692,
          2229,  2226,  1007,  8299,  1024,  1013,  1013,  1056,  1012,  2522,
          1013,  1044,  2692,  6777, 19279,  2497, 13088,  4160,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

## Task 5: Setting up BERT Pretrained Model

In [43]:
# importing pretrained BERT model
# we're treating each tweet as its own unique sequence 
# so one sequence will be classified into one of six classes
from transformers import BertForSequenceClassification

In [44]:
# This step is to redefine the architecture to include parts that we need
model = BertForSequenceClassification.from_pretrained(
    # reason for using this is computationally efficient 
    # it is a small version 
    'bert-base-uncased',

    # how many output labels the final layout of BERT must have 
    # will be able to classify 
    num_labels = len(label_dict),

    # we dont actually want any unnecessary outputs from the model
    output_attentions = False,

    # we don't care the output of the hidden state which is 
    # the state just before the prediction 
    # this might be useful for encoding situations, but for us, we don't really need them
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
# now investigating the model
# by printing out the model, we can see that the model is huge
# from the huggingface info, https://huggingface.co/transformers/pretrained_models.html 
# the model is 12-layer, 768-hidden, 12-heads, 110M parameters. Trained on lower-cased English text.
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [46]:
# type of the model
type(model)

transformers.modeling_bert.BertForSequenceClassification

In [47]:
# now look at model parameters
model.parameters()

<generator object Module.parameters at 0x7fce910b2570>

## Task 6: Creating Data Loader

In [48]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [49]:
# normally we can use the batch size 32
# however, in this case, we use 4 because of 
# the limited resources including memory and hardware
batch_size = 32  #32

dataloader_train = DataLoader(
    dataset_train,
    # this is preventing model to learn sequentially differences when training
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    # setting back the batch size to 32 
    # since we don't need to do any back propagation
    batch_size=32
)

In [50]:
type(dataloader_train)

torch.utils.data.dataloader.DataLoader

In [51]:
# the number of dataloader_train, 315, is smaller than the number of dataset_train, 1258
# this could be because the sampling process which gets smaller size for the dataloader_train
len(dataloader_train)

40

In [52]:
# there is no direct way to indexing the dataloader
# thus using for loop to get the items
# in this case, the item's type is a list
n = 0
for i in dataloader_train:
    if n <= 2:
        print(type(i))
        n+=1

<class 'list'>
<class 'list'>
<class 'list'>


In [53]:
# now we investigate number of elements in each list
# there are 3 elements in each list
n = 0
for i in dataloader_train:
    if n <= 2:
        print(len(i))
        n+=1

3
3
3


In [54]:
# now we print out only one item in dataloader by setting n <= 0
# the items has a form of 3 tensors including input_ids, attention_mask & labels
n = 0
for i in dataloader_train:
    if n <= 0:
        print(i)
        n+=1

[tensor([[ 101, 1996, 3819,  ...,    0,    0,    0],
        [ 101, 1030, 3680,  ...,    0,    0,    0],
        [ 101, 1045, 2074,  ...,    0,    0,    0],
        ...,
        [ 101, 2256, 1001,  ...,    0,    0,    0],
        [ 101, 1030, 9902,  ...,    0,    0,    0],
        [ 101, 1012, 1030,  ...,    0,    0,    0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0])]


In [55]:
# now we print out only one item in dataloader by setting n <= 0
# the items has a form of 3 tensors including input_ids, attention_mask & labels
n = 0
for i in dataloader_train:
    if n <= 0:
        print(i[0])
        print(i[1])
        print(i[2])
        n+=1

tensor([[  101,  1030, 14753,  ...,     0,     0,     0],
        [  101,  1030,  2329,  ...,     0,     0,     0],
        [  101,  1045,  2018,  ...,     0,     0,     0],
        ...,
        [  101,  1030,  2120,  ...,     0,     0,     0],
        [  101,  1037,  2210,  ...,     0,     0,     0],
        [  101,  6854,  1996,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 4])


In [56]:
# now investigate the dimension of each of the elements in one item
# we can see that the first element, input_ids, has a dimension of 4x256 where 4 is the number of batch_size and 256 is the number of the chosen max_length
# similarly, the attention_mask, the second element also has a dimension of 4x256
# finally, the labels have a dimension of 4 as the number of batch_size
n = 0
for i in dataloader_train:
    if n <= 0:
        print(i[0].shape)
        print(i[1].shape)
        print(i[2].shape)
        n+=1

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32])


## Task 7: Setting Up Optimizer and Scheduler

In [57]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [58]:
# now we do optimisation, more info from this post https://huggingface.co/transformers/main_classes/optimizer_schedules.html
# in this case, we choose AdamW which is 
optimizer = AdamW(
    # model's parameters - Iterable of parameters to optimize or dictionaries defining parameter groups.
    model.parameters(),

    # learning rate
    lr=1e-5, #2e-5 > 5e-5

    # Is a very small number to prevent any division by zero in the implementation (e.g. 10E-8).
    # source: https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
    eps=1e-8
)

In [59]:
print(type(optimizer))
optimizer

<class 'transformers.optimization.AdamW'>


AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.0
)

In [60]:
epochs = 10

# Create a schedule with a learning rate that decreases linearly from 
# the initial lr set in the optimizer to 0, after a warmup period during 
# which it increases linearly from 0 to the initial lr set in the optimizer.

# Warm up steps is a parameter which is used to lower the learning rate 
# in order to reduce the impact of deviating the model 
# from learning on sudden new data set exposure.
# source: https://stackoverflow.com/questions/60120043/optimizer-and-scheduler-for-bert-fine-tuning
scheduler = get_linear_schedule_with_warmup(
    # The optimizer for which to schedule the learning rate.
    optimizer,

    # The number of steps for the warmup phase
    num_warmup_steps=0,

    # The total number of training steps.
    num_training_steps=len(dataloader_train)*epochs
)

In [61]:
len(dataloader_train)*epochs

400

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in this [tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)


In [62]:
import numpy as np

In [63]:
from sklearn.metrics import f1_score

In [64]:
# predictions will come in a form of a list-like of 6 values representing probabilities distribution
# preds = [0.9, 0.05, 0.05, 0, 0, 0]

# but we want it to become
# preds = [1, 0, 0, 0, 0, 0]

In [65]:
def f1_score_func(preds, labels):
    # we want the flat value of preditions, we don't want the list of lists
    # thus we use flatten()
    preds_flat = np.argmax(preds, axis=1).flatten()

    # we do the same things with labels 
    labels_flat = labels.flatten()

    # then we use f1_score from sklearn to calculate with weighted average
    return f1_score(labels_flat, preds_flat, average='weighted')

In [66]:
# for example, taking all labels of class 5 and let see how many class 5 being predicted
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')


## Task 9: Creating our Training Loop

In [67]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [69]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []    
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [70]:
# the training loops
for epoch in tqdm(range(1, epochs + 1)):
    
    # stating model in the training mode
    model.train()

    # setting training loss to zero
    # each epoch getting average training loss then we add each batch's loss onto this variable
    loss_train_total = 0

    # using progress bar method from tqdm to see how many batches have been trained and how many to go
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch),
                        # leave = False --> overwrite itself after each epoch
                        leave = False, 
                        disable=False)
    
    # refering to the progress_bar above, this is the wrapping method 
    # which means progress_bar is now the dataloader_train 
    # it is illustrated in the function demo below
    for batch in progress_bar:
        
        # the standard pytorch procedure, need to set zero gradient for model
        model.zero_grad()

        # reminder: elements of dataloader_train having 3 different variables in the tuple form
        # now we try to ensure each of individual items of a single tuple on the correct device whether cuda or cpu
        batch = tuple(b.to(device) for b in batch)

        # now we pull out the inputs in the format of dictionary, our inputs is what to go to the model
        inputs = {
            'input_ids':       batch[0],
            'attention_mask':  batch[1],
            'labels':          batch[2]
        }
        
        # now we get the output coming from the input and the model
        outputs = model(**inputs)

        # the first output of the BERT training model is the loss
        # thus we assign the model loss to the variable loss 
        loss = outputs[0]

        # then we add up the loss to loss_train_total
        loss_train_total += loss.item()

        # then we do back propagation on loss
        loss.backward()

        # clipping gradients into a norm value, in this case it is 1.0 
        # to prevent the gradients vanishing or exploding
        torch.nn.utils.clip_grad_value_(model.parameters(), 1.0)

        # to update optimizer
        optimizer.step()

        # to update scheduler
        scheduler.step()

        # update progress_bar to show the loss per batch
        progress_bar.set_postfix({'training loss':  '{:.3f}'.format(loss.item()/len(batch))})

    # save the model per epoch
    torch.save(model.state_dict(), directory_path + f'/models/BERT_ft_{epoch}.model')

    # report few more things
    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss:  {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')


Epoch 1
Training loss:  1.0725305318832397
Validation loss: 0.7962499175752912
F1 score (weighted): 0.7136675190820192



Epoch 2
Training loss:  0.7259105630218983
Validation loss: 0.6924787291458675
F1 score (weighted): 0.7385602518843621



Epoch 3
Training loss:  0.5748166345059872
Validation loss: 0.6455750209944588
F1 score (weighted): 0.7684224498198171



Epoch 4
Training loss:  0.4956930406391621
Validation loss: 0.5946802794933319
F1 score (weighted): 0.7790288504974604



Epoch 5
Training loss:  0.4328852526843548
Validation loss: 0.5864090940782002
F1 score (weighted): 0.7816587566858171



Epoch 6
Training loss:  0.3896180383861065
Validation loss: 0.5758848403181348
F1 score (weighted): 0.780976441609297



Epoch 7
Training loss:  0.3513915497809649
Validation loss: 0.5923969873360225
F1 score (weighted): 0.7862480072500582



Epoch 8
Training loss:  0.35097981058061123
Validation loss: 0.5779805545295987
F1 score (weighted): 0.7858849957352656



Epoch 9
Training loss:  0.3247888036072254
Validation loss: 0.5813159389155251
F1 score (weighted): 0.7954993058692611



Epoch 10
Training loss:  0.3161234274506569
Validation loss: 0.5795356120382037
F1 score (weighted): 0.7954993058692611



## Task 10: Loading and Evaluating our Model

In [74]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    # how many output labels the final layout of BERT must have 
    # will be able to classify 
    num_labels = len(label_dict),

    # we dont actually want any unnecessary outputs from the model
    output_attentions = False,

    # we don't care the output of the hidden state which is 
    # the state just before the prediction 
    # this might be useful for encoding situations, but for us, we don't really need them
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [76]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [77]:
model.load_state_dict(
    torch.load(
        directory_path + '/models/BERT_ft_10.model',
        map_location=torch.device(device)
    )
)

<All keys matched successfully>

In [78]:
loss_val_avg, predictions, true_vals = evaluate(dataloader_val)

In [79]:
accuracy_per_class(predictions,true_vals)

Class: happy
Accuracy: 0.9532163742690059

Class: not-relevant
Accuracy: 0.6875

Class: angry
Accuracy: 0.0

Class: disgust
Accuracy: 0.0

Class: sad
Accuracy: 0.0

Class: surprise
Accuracy: 0.0





**function demo for the chuck of code above**

In [71]:
# tqdm function is to show the progess bar as each training epoch going through
tqdm(range(1,epochs+1))

  0%|<bar/>| 0/10 [00:00<?, ?it/s]

In [72]:
epoch = 3
tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch))

Epoch 3:   0%|<bar/>| 0/40 [00:00<?, ?it/s]

In [73]:
# from this, we can see that the prog_bar is now the dataloader_train
prog_bar = tqdm(dataloader_train)
for batch1 in prog_bar:
    print(batch1)

[tensor([[  101,  4957,  3972,  ...,     0,     0,     0],
        [  101,  1015,  2733,  ...,     0,     0,     0],
        [  101,  1030,  8223,  ...,     0,     0,     0],
        ...,
        [  101,  2559,  2830,  ...,     0,     0,     0],
        [  101,  1012,  1030,  ...,     0,     0,     0],
        [  101,  1030, 21113,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([1, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 2, 0, 0, 0, 0, 4])]
[tensor([[  101,  4760,  2115,  ...,     0,     0,     0],
        [  101,  1030, 17623,  ...,     0,     0,     0],
        [  101,  6284,  2660,  ...,     0,     0,     0],
        ...,
        [  101,  1030, 11937,  ...,     0,     0,     0],
        [  101,  4292,  2039,  ...,     0,     0,     0],

## 